In [2]:
# std lib
from getpass import getpass

# 3rd party
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from astropy.utils.data import download_file  #import file from URL
from scipy.stats import binned_statistic_2d
%matplotlib inline
from astropy.table import Table
import pandas as pd

# Data Lab
from dl import authClient as ac, queryClient as qc, storeClient as sc
from dl.helpers.utils import convert

import astropy
from astropy import units as u
from astropy.coordinates import SkyCoord

print('Done importing')

Done importing


In [51]:
c = SkyCoord(["21h44m25s -40d54m0s","21h33m24s -54d44m54s","21h49m23s -52d58m15s"], frame='icrs')

RA180924 = c[0].ra.degree
RA191001 = c[1].ra.degree
RA181112 = c[2].ra.degree

DEC180924 = c[0].dec.degree
DEC191001 = c[1].dec.degree
DEC181112 = c[2].dec.degree

print(RA180924, DEC180924)
print(RA191001, DEC191001)
print(RA181112, DEC181112)

326.10416666666663 -40.9
323.34999999999997 -54.748333333333335
327.3458333333333 -52.97083333333334


In [87]:
data=pd.read_csv("GSfrb191001.csv")

In [88]:
data

,UCAC4,RAJ2000,DEJ2000,pmRA,pmDE,Vmag,rmag,of
0,172-204234,322.551951,-55.715890,5.4,-7.2,13.423,13.218,0
1,172-204292,322.913947,-55.600122,14.2,-7.2,13.082,12.918,0
2,172-204344,323.356802,-55.701481,23.8,9.3,13.127,12.966,0
3,172-204402,323.844523,-55.654468,10.8,-22.8,13.195,12.998,0
4,172-204419,323.959608,-55.740994,-7.8,5.9,13.267,13.133,0
...,...,...,...,...,...,...,...,...
63,181-218558,323.081019,-53.859105,-8.1,-8.2,13.011,12.886,0
64,181-218588,323.315909,-53.927839,18.6,-8.1,13.359,13.154,0
65,181-218635,323.620465,-53.898940,6.7,-3.8,13.420,13.178,0
66,181-218703,324.051089,-53.900709,23.4,-23.4,13.498,13.290,0


In [89]:
iin = np.where((np.logical_not(np.isnan(np.array(data['rmag'])))) & (data['of']==0))

In [90]:
dt = np.dtype([('name','U15'),('RA.h','i1'),('RA.m','i1'),('RA.s','f8'),('dec.d','i1'),('dec.m','i1'),('dec.s','f8'),
              ('type','U5'),('prt','i1'),('mag','f4'),('Prog_ID','i1'),('PM_ra','f8'),('PM_dec','f8'),('cmt','U50')])

N=len(iin[0])
arr1 = np.zeros(N,dtype=dt)
print(N)

68


In [93]:
arr1[:]['name'] = np.array(data['UCAC4'])[iin]
c = SkyCoord(np.array(data['RAJ2000'])[iin]*u.degree, np.array(data['DEJ2000'])[iin]*u.degree, frame='icrs')
arr1[:]['RA.h'] = np.rint(c[:].ra.hms[0])
arr1[:]['RA.m'] = np.rint(c[:].ra.hms[1])
arr1[:]['RA.s'] = c[:].ra.hms[2]
arr1[:]['dec.d'] = np.rint(c[:].dec.dms[0])
arr1[:]['dec.m'] = np.rint(abs(c[:].dec.dms[1]))
arr1[:]['dec.s'] = abs(c[:].dec.dms[2])
arr1[:]['type'] = np.full(N,'F')
arr1[:]['prt'] = np.full(N,9)
arr1[:]['mag'] = np.array(data['rmag'])[iin]
arr1[:]['Prog_ID'] = np.full(N,0)
arr1[:]['PM_ra'] = np.array(data['pmRA'])[iin] * 0.001
arr1[:]['PM_dec'] = np.array(data['pmDE'])[iin] * 0.001

In [94]:
arr1

array([('172-204234', 21, 30, 12.468216, -55, 42, 57.20328, 'F', 9, 13.218, 0,  0.0054, -0.0072, ''),
       ('172-204292', 21, 31, 39.34716 , -55, 36,  0.4392 , 'F', 9, 12.918, 0,  0.0142, -0.0072, ''),
       ('172-204344', 21, 33, 25.63236 , -55, 42,  5.33016, 'F', 9, 12.966, 0,  0.0238,  0.0093, ''),
       ('172-204402', 21, 35, 22.68552 , -55, 39, 16.08624, 'F', 9, 12.998, 0,  0.0108, -0.0228, ''),
       ('172-204419', 21, 35, 50.305848, -55, 44, 27.57912, 'F', 9, 13.133, 0, -0.0078,  0.0059, ''),
       ('172-204423', 21, 35, 59.896872, -55, 37, 25.81104, 'F', 9, 13.301, 0,  0.0212, -0.0065, ''),
       ('172-204441', 21, 36, 44.602536, -55, 37, 13.30104, 'F', 9, 13.324, 0,  0.0332, -0.0114, ''),
       ('173-215018', 21, 29, 27.70788 , -55, 31, 13.39212, 'F', 9, 12.874, 0,  0.0029, -0.0154, ''),
       ('173-215047', 21, 30, 15.798   , -55, 24, 46.8432 , 'F', 9, 13.262, 0,  0.0039,  0.0054, ''),
       ('173-215146', 21, 33, 24.421488, -55, 34, 31.59732, 'F', 9, 12.913, 0, -0.

In [86]:
f = open("FRB191001.txt",'a')
for i in range(N):
    f.write('{:0>11s}   {:0>+3d} {:0>2d} {:0>6.3f}   {:0>+3d} {:0>2d} {:0>6.3f}   {:<5s}{:<4d}{:0>6.3f}   {:<4d}{:0<+7.4f} {:0<+7.4f}   {:<30s}'.format(
    arr1[i]['name'],arr1[i]['RA.h'],arr1[i]['RA.m'],arr1[i]['RA.s'],arr1[i]['dec.d'],arr1[i]['dec.m'],arr1[i]['dec.s'],
    arr1[i]['type'],arr1[i]['prt'],arr1[i]['mag'],arr1[i]['Prog_ID'],arr1[i]['PM_ra'],arr1[i]['PM_dec'],arr1[i]['cmt']))
    f.write('\n') 
f.close()